# MLflow Quick Start Notebook (Databricks Hosted Tracking Server)
This is a Quick Start notebook based on MLflow’s tutorial. In this tutorial, we’ll: Connect our notebook to an MLflow Tracking Server that is hosted by Databricks; Log metrics, parameters, models to show how you can record arbitrary outputs from your MLflow job, then view our results on the MLflow tracking UI.

This notebook uses one of the Contoso datasets in scikit-learn and predicts the price of a laptop based on its description. It uses the scikit-learn Random Forest regression model, 

A good reference for MLflow in general is Matei’s Spark Summit 2018 Keynote.

### REQUIREMENT

Create MLFlow Experiment and Define Experiment Name and ID

In [3]:
experimentName = "/Users/silviu.tofan@databricks.com/Ready for Git/Contoso-retail/4-ML/contoso_models"
experiment_id = "4289906168992634"

In [4]:
#Read information about our products and transform it to pandas
df = spark.read.parquet("/mnt/databricks-workshop-datasets/Contoso-retail/initech/productsFull")
pandasDF = df.toPandas()

In [5]:
#Look at the type of info we have
pandasDF.head()

Out[ 3 ]: 
 ProductID Name ProductNumber \
0 7724 "HP Spectre x360 2-in-1 13.3" 4K Ultra HD Touc... L-0001-BU 
1 7725 Microsoft Surface Pro – 12.3""" L-0002-YE 
2 7726 Microsoft Surface Book 2 - 13.5""" L-0003-OR 
3 7727 Dell XPS 2-in-1 13.3""" L-0004-GR 
4 7728 Lenovo Yoga 920 2-in-1 13.9""" L-0005-SI 

 Color StandardCost ListPrice Size Weight ProductCategoryID \
0 blue 1499.99 1499.99 12.3" None 2 
1 yellow 1299.99 1299.99 12.3" None 2 
2 orange 1499.99 1499.99 13.5" None 2 
3 gray 1949.99 1949.99 12.3" None 2 
4 silver 1799.99 1799.99 None None 2 

 ProductModelID SellStartDate SellEndDate DiscontinuedDate \
0 1 2018-02-06 2018-02-06 None 
1 2 2018-02-06 2018-02-06 None 
2 3 2018-02-06 2018-02-06 None 
3 4 2018-02-06 2018-02-06 None 
4 5 2018-02-06 2018-02-06 None 

 ThumbNailPhoto ThumbnailPhotoFileName \
0 [82, 48, 108, 71, 79, 68, 108, 104, 85, 65, 65... no image available 
1 [82, 48, 108, 71, 79, 68, 108, 104, 85, 65, 65... no image available 
2 [82, 48, 108, 71, 79, 68, 108, 104, 85, 65, 65... no image available 
3 [82, 48, 108, 71, 79, 68, 108, 104, 85, 65, 65... no image available 
4 [82, 48, 108, 71, 79, 68, 108, 104, 85, 65, 65... no image available 

 ModifiedDate 
0 2018-02-06 
1 2018-02-06 
2 2018-02-06 
3 2018-02-06 
4 2018-02-06

# Let's apply TFIDF on our product name to see if we can predict the price purely based off of it

<img src=https://cdn-images-1.medium.com/max/2000/1*q3qYevXqQOjJf6Pwdlx8Mw.png
      />

https://en.wikipedia.org/wiki/Tf%E2%80%93idf

In [7]:
#Let's build a prediction based on the description of the product

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas

#Create TFIDF 
v = TfidfVectorizer()
x = v.fit_transform(pandasDF['Name'])

#Add our values to the initial dataframe
df1 = pandas.DataFrame(x.toarray(), columns=v.get_feature_names())
res = pandas.concat([pandasDF, df1], axis=1)

#Create our training and testing dataframes
X = res.iloc[:, 17:]
y = res['ListPrice']

In [8]:
from sklearn.ensemble import RandomForestRegressor
import mlflow
import mlflow.sklearn
import pandas
import numpy
from sklearn.model_selection import train_test_split

print("MLflow Version:",mlflow.version.VERSION)
mlflow_client = mlflow.tracking.MlflowClient()

mlflow.set_experiment(experimentName)

print("experiment_id:",experiment_id)
print("experiment_name:",experimentName)

with mlflow.start_run():

  #Send the dataframes into Pandas
  trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.33, random_state=42)
  
  #Define number of estimators and log them
  n_est = 1000
  mlflow.log_param("n_est", n_est)
  
  #Train the model
  rfsk = RandomForestRegressor(n_estimators = n_est, random_state = 42)
  rfsk.fit(trainX, trainY)

  #Make the predictions
  predictions = rfsk.predict(testX)
  predDF = pandas.DataFrame(testY[:])
  predDF['pred'] = predictions
  predDF['errors'] = predDF["ListPrice"] - predDF['pred']

  #Get standard deviation of errors
  stdv = predDF['errors'].std()

  #Mean absolute error of predictions
  mae = predDF['errors'].abs().mean()
  
  #Log to mlflow
  mlflow.log_metric("mae", mae)
  mlflow.log_param("std", stdv)
  mlflow.sklearn.log_model(rfsk, "model1sklearn")

/databricks/python/lib/python3.5/site-packages/mlflow/utils/environment.py:26: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
 env = yaml.load(_conda_header)
MLflow Version: 0.9.0
experiment_id: 4289906168992634
experiment_name: /Users/silviu.tofan@databricks.com/Ready for Git/Contoso-retail/4-ML/contoso_models

## Next Step

[5-01 Simple Pipeline without Delta]($../5-Delta/5-01 Simple Pipeline without Delta)

&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>